> Premature optimization is the root of all evil - Donald Knuth

How to measure the performance objectively?

- `timeit` - measure the runtime performance of code snippets
- `cProfile` - analyze performance of entire functions.

# `timeit` module

In [11]:
import random
import bisect

In [12]:
%%timeit
# With notebook

random.randint(1, 100)

1.2 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
import timeit

timeit.timeit('random.randint(1, 100)', 'import random', number=1_000_000)

1.266308072999891

# `cProfile` module


In [14]:
import cProfile

def add_random_number(n=1000):
	total = 0
	for i in range(n):
		r = random.randint(1, 100)
		total += r
	return total

In [15]:
%%prun

add_random_number()

In [16]:
cProfile.run('add_random_number()')

         5284 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.003    0.003 <ipython-input-14-f305c7b5d1c9>:3(add_random_number)
        1    0.000    0.000    0.003    0.003 <string>:1(<module>)
     1000    0.001    0.000    0.001    0.000 random.py:237(_randbelow_with_getrandbits)
     1000    0.001    0.000    0.002    0.000 random.py:290(randrange)
     1000    0.001    0.000    0.003    0.000 random.py:334(randint)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
     1000    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
     1280    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}




`cProfile` is to help determine which parts of your code contribute to
the total program runtime.
So we could focus our optimization efforts on parts that contributes
the largest bottleneck and ignore parts of your code that don't.

## Finding a needle in a haystack, is my needle there?

In [17]:
needle = 89_998_999
haystack = list(range(100_000_000))

In [18]:
def scan(needle, haystack):
	for i in haystack:
		if i == needle:
			return True
	return False


cProfile.run("scan(needle, haystack)")

         4 function calls in 7.888 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    7.887    7.887    7.887    7.887 <ipython-input-18-759a2c6e9552>:1(scan)
        1    0.000    0.000    7.887    7.887 <string>:1(<module>)
        1    0.000    0.000    7.888    7.888 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [19]:
def binary_search(needle, haystack):
	if not len(haystack):
		return False
	start_index = 0
	end_index = len(haystack) - 1

	while start_index <= end_index:
		mid_index = (start_index + end_index) // 2
		if haystack[mid_index] == needle:
			return True
		elif needle < haystack[mid_index]:
			end_index = mid_index - 1
		elif needle > haystack[mid_index]:
			start_index = mid_index + 1

	return False



def bisect_search(needle, haystack):
	index = bisect.bisect_left(haystack, needle)
	return index < len(haystack) and haystack[index] == needle

In [20]:
cProfile.run("binary_search(needle, haystack)")
cProfile.run("bisect_search(needle, haystack)")


         6 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-19-c3d845412806>:1(binary_search)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         6 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-19-c3d845412806>:20(bisect_search)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_left}
        1    0.000    0.000    0.000    0.000 {

References:

- https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html


